In [ ]:
import pandas as pd
import torch
import sys
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch.utils.data import random_split
from sklearn.preprocessing import MinMaxScaler


sys.path.append("../")
from src.architectures.lstm.classifier import LSTMClassifier
from src.architectures.lstm.autoencoder import LSTMAutoEncoder
from src.dataset import ExerciseDataset
from src.trainer import Trainer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = torch.manual_seed(0)

In [ ]:
squat_angles_df = pd.read_csv("../data/angles/squat.csv")
scaler = MinMaxScaler()
columns_to_normalize = [ "left_knee",	"right_knee",	"right_arm",	"left_arm",	"left_hip",	"right_hip"]
squat_angles_norm_df = pd.DataFrame(scaler.fit_transform(squat_angles_df[columns_to_normalize]), columns=columns_to_normalize)
squat_angles_df[columns_to_normalize] = squat_angles_norm_df

squat_angles_df.head()

In [ ]:
squat_angles_dataset = ExerciseDataset(squat_angles_df, representation="angles")
dataset_length = len(squat_angles_dataset)

In [ ]:
train_squat_angles_dataset, val_squat_angles_dataset = random_split(
    squat_angles_dataset,
    [int(0.95 * dataset_length), dataset_length - int(0.95 * dataset_length)],
    generator,
)

In [ ]:
batch_size = 8

train_squat_angles_dl = DataLoader(
    train_squat_angles_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=ExerciseDataset.resample_batch,
)

val_squat_angles_dl = DataLoader(
    val_squat_angles_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=ExerciseDataset.resample_batch,
)

In [ ]:
NUM_ANGLES = 6

LATENT_DIM = 512
NUM_LAYERS = 8
HIDDEN_SIZE = 1024

In [ ]:
angles_lstm_autoencoder = LSTMAutoEncoder(NUM_ANGLES, HIDDEN_SIZE, LATENT_DIM, NUM_LAYERS)
loss_fn = torch.nn.MSELoss(reduction="sum")
lr = 0.02
optimizer = torch.optim.Adam(angles_lstm_autoencoder.parameters(), lr)

In [ ]:
trainer = Trainer(  
    angles_lstm_autoencoder, train_squat_angles_dl, val_squat_angles_dl, loss_fn, optimizer, device
)
results = trainer.train_autoencoder(num_epochs=100, weights_path="../models/angles_lstm_autoencoder.pt")

In [ ]:
plt.plot([result["train_loss"] for result in results], label="train")

In [ ]:
angles_lstm_autoencoder.load_state_dict(torch.load("../models/angles_lstm_autoencoder.pt", map_location=torch.device('cpu')))

In [ ]:
dl_iter = iter(train_squat_angles_dl)
X, _, _ = next(dl_iter)
X = X[1].to(device).unsqueeze(0)
X_hat = angles_lstm_autoencoder(X, [X.shape[1]])

plt.plot(X.squeeze())


In [ ]:
plt.plot(X_hat.detach().numpy().squeeze())